In [1]:
import pygsheets
import pandas as pd
import numpy as np
import datetime as dt
import time
import filepath
import pandas as pd
import glob
import os
import numpy as np
import filepaths
import send_email 
import pygsheets
import infrastructure
import datetime
import math
import datetime
from datetime import timedelta  
import re
import warnings
import statistics
from datetime import date
from calendar import monthrange
from dateutil.relativedelta import relativedelta
import xlsxwriter
import send_email
from colorama import Fore, Style
import matplotlib.pyplot as plt
import seaborn as sns
import filepath 
import datetime as dt

In [2]:
def gap_vert_series(row,n=5,z=2):
    if ', ' in row['index_s']:
        sloping=pd.Series([float(date) for date in row.CTR.split(', ')], index=[int(i) for i in row.index_s.split(', ')],name='CTR')
        sloping2=pd.Series([float(date) for date in row.eCPM.split(', ')], index=[int(i) for i in row.index_s.split(', ')],name='eCPM')
        sloping3=pd.Series([float(date) for date in row.JPCTR.split(', ')], index=[int(i) for i in row.index_s.split(', ')],name='JPCTR')
        #sloping4=pd.Series([float(date) for date in row.vgap.split(', ')], index=[int(i) for i in row.index_s.split(', ')],name='vgap')#.diff().dt.days.fillna(0).to_dict()
        gapping=pd.Series([dt.datetime.strptime(date, '%m.%d.%y').date() for date in row.Datestring.split(', ')], index=[int(i) for i in row.index_s.split(', ')],name='gapping')#.diff().dt.days.fillna(0).to_dict()
        sloping=pd.concat([sloping,gapping,sloping2,sloping3],axis=1)
        sloping=pd.DataFrame(sloping).sort_index()
        # Calculate CTR slop by comparing the current drop with previous drop, the max/median/min CTR in the last 5 drops( if there're less than 5 drops, we need a least 2 drops)
        sloping['Offer recent cslope']=np.log(sloping['CTR']) - np.log(sloping['CTR'].shift(1)) # CTR slope between the current drop and previous drop 
        #sloping['Offer CTR recent max slope']=sloping['CTR']/sloping['CTR'].shift(periods=1).rolling(n,min_periods=z).max()-1# the max CTR in the last n item with at least Z values 
        sloping['Offer CTR recent median slope']=sloping['CTR']/sloping['CTR'].shift(periods=1).rolling(n,min_periods=z).median()-1 # the median CTR in the last n item with at least Z values 
        #sloping['Offer CTR recent min slope']=sloping['CTR']/sloping['CTR'].shift(periods=1).rolling(n,min_periods=z).min()-1 # the min CTR in the last n item with at least Z values 
        # same slope calculation for open Rate 
        sloping['Offer recent jpcslope']=np.log(sloping['JPCTR']) - np.log(sloping['JPCTR'].shift(1))
        #sloping['Offer openrate recent max slope']=sloping['openrate']/sloping['openrate'].shift(periods=1).rolling(n,min_periods=z).max()-1
        sloping['Offer jpctr recent median slope']=sloping['JPCTR']/sloping['JPCTR'].shift(periods=1).rolling(n,min_periods=z).median()-1
        #sloping['Offer openrate recent min slope']=sloping['openrate']/sloping['openrate'].shift(periods=1).rolling(n,min_periods=z).min()-1
        
        #sloping['Offer recent gap']=sloping['gapping'].diff().dt.days # lastr 2 drops gap 
        #sloping['Offer trending gap']=sloping['Offer recent gap'].shift(periods=1).rolling(n,min_periods=z).mean() # average gappping in the last 5 drops ( if there're less tha 5 drops, we need a least 2 drops)
        #sloping['Offer trending period']=sloping['Offer recent gap'].rolling(n,min_periods=z).sum() 
        #sloping['Vertical trending gap']=sloping['vgap'].shift(periods=1).rolling(n,min_periods=z).mean()
        
        sloping['Offer eCPM recent slope']=np.log(sloping['eCPM']) - np.log(sloping['eCPM'].shift(1))
        #sloping['Offer eCPM prior slope']=sloping['Offer eCPM recent slope'].shift(periods=1)
        #sloping['Offer eCPM trending slope']=sloping['Offer eCPM recent slope'].rolling(n,min_periods=1).sum() # rolling sum to see whether the offer is trening up (>0) or trending down (<0) in overall
        #sloping['Offer eCPM recent max slope']=sloping['eCPM']/sloping['eCPM'].shift(periods=1).rolling(n,min_periods=z).max()-1
        sloping['Offer eCPM recent median'] = sloping['eCPM'].shift(periods=1).rolling(n,min_periods=z).median()
        sloping['Offer eCPM recent median slope']=sloping['eCPM']/sloping['eCPM'].shift(periods=1).rolling(n,min_periods=z).median()-1
        #sloping['Offer eCPM recent min slope']=sloping['eCPM']/sloping['eCPM'].shift(periods=1).rolling(n,min_periods=z).min()-1
        
    else:
        sloping=pd.Series(dtype=float)
    small_frame.update(sloping) #save all datapoints in small frame 
    

In [3]:
#%%

t2=time.time()
sms = pd.read_csv(filepath.output_folder+'SS_LC_merged_data.csv')
df = infrastructure.transform_sms_df(sms)
all_send_strategy_df = df.copy()
df = df[~df['Send Strategy'].isin(['PT','RT','IT','OT','W','MI','Mi'])]
#testMaster = scheduleMaster[scheduleMaster['Send Strategy'].isin(['PT','RT','IT','OT','CT'])]
#miningMaster = scheduleMaster[scheduleMaster['Send Strategy']=='MI']
#scheduleMaster = scheduleMaster[~(scheduleMaster.index.isin(testMaster.index) | scheduleMaster.index.isin(miningMaster.index))] #create index s time series sorted index
df['Datestring']=df['Date'].dt.strftime('%m.%d.%y') #create datestring variable 
df = df.groupby(['SC_DP&Pub','Hitpath Offer ID','Datestring','Date','Send Strategy']).agg({'Revenue':'sum','Clicks':'sum','Delivered':'sum','Jump Page Clicks':'sum','Opportunity Cost':'sum','Optout':'sum','Cost':'sum'}).reset_index()
df['CTR']=(df['Clicks']/df['Delivered']).astype(str)
df['eCPM']=(df['Revenue']*1000/df['Delivered']).astype(str)
df['JPCTR']=(df['Jump Page Clicks']/df['Delivered']).astype(str)
df['optout rate']=(df['Optout']/df['Delivered']).astype(str)

scheduleMaster = df.copy()
scheduleMaster['index_g']=scheduleMaster.index #create index g used for final dataframe update
scheduleMaster['index_g'] = scheduleMaster['index_g'].astype(str)
scheduleMaster['index_s']=(scheduleMaster.sort_values(['Date']).reset_index().index).astype(str)

#%%
small_frame=scheduleMaster[['index_g','index_s']]
small_frame['index_s']=small_frame['index_s'].astype(int)
small_frame.index=small_frame['index_s']
""" 
new_metrics=['Offer recent cslope',
       'Offer CTR recent max slope', 'Offer CTR recent median slope','Offer CTR recent min slope',
       'Offer recent oslope', 'Offer openrate recent max slope','Offer openrate recent median slope', 
       'Offer openrate recent min slope', 'Offer recent gap',
       'Offer trending gap', 'Offer trending period', 'Vertical trending gap',
       'Offer eCPM recent slope', 'Offer eCPM prior slope',
       'Offer eCPM trending slope', 'Offer eCPM recent max slope','Offer eCPM recent median slope',
       'Offer eCPM recent min slope']
"""
# rewrite new_metrics with only recent and median slope 
new_metrics=['Offer recent cslope', 'Offer CTR recent median slope','Offer recent jpcslope','Offer jpctr recent median slope', 
        'Offer eCPM recent slope','Offer eCPM recent median', 'Offer eCPM recent median slope']

# for i in the_list: #add all new stats
#     new_metrics.extend([f'{i} recent slope',f'{i} prior slope',f'{i} trending slope',f'{i} recent max slope',f'{i} recent min slope',
#              f'{i} recent gap',f'{i} prior gap',f'{i} trending gap',f'{i} trending period',f'tscore {i}',f'pval {i}'])
#     new_metrics.extend([f'{i} segment recent slope',f'{i} segment prior slope',f'{i} segment trending slope',f'{i} segment recent max slope',f'{i} segment recent min slope',
#                   f'{i} segment recent gap',f'{i} segment prior gap',f'{i} segment trending gap',f'{i} segment trending period',f'tscore segment {i}',f'pval segment {i}'])
#     new_metrics.extend([f'{i} eCPM recent slope',f'{i} eCPM prior slope',f'{i} eCPM trending slope',f'{i} eCPM recent max slope',f'{i} eCPM recent min slope',
#              f'{i} eCPM recent gap',f'{i} eCPM prior gap',f'{i} eCPM trending gap',f'{i} eCPM trending period',f'tscore {i} eCPM',f'pval {i} eCPM'])
small_frame[new_metrics]=np.nan
df[new_metrics]=np.nan



vert_data=scheduleMaster.groupby(['SC_DP&Pub','Hitpath Offer ID']).agg({'index_s':', '.join,'CTR':', '.join,'Datestring':', '.join,'eCPM':', '.join,'JPCTR':', '.join}) #creates dataframe with all indexes, CTRs and Dates captured by the groupby
vert_data.apply(gap_vert_series,axis=1)

#%%
small_frame['index_g']=small_frame['index_g'].astype(int)
small_frame.index=small_frame['index_g']
df.update(small_frame)
df['CTR']= df['CTR'].astype(float)
df['eCPM']= df['eCPM'].astype(float)
df['JPCTR'] = df['JPCTR'].astype(float)  
df['optout rate']= df['optout rate'].astype(float)
df['Opportunity Cost eCPM on dataset-level'] = df['Opportunity Cost']*1000/df['Delivered'] 


D:\Users\lilig\AppData\Local\Temp\1\ipykernel_10856\3438278588.py:4: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,14,16,25,26,27,28,29,30,31,32,33,34,35,36,43,47,50,54) have mixed types. Specify dtype option on import or set low_memory=False.
  sms = pd.read_csv(filepath.output_folder+'SS_LC_merged_data.csv')
D:\Users\lilig\Desktop\github\SMS_Report\Daily_Reporting\infrastructure.py:60: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pygsheets\worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty strin

D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-pa

D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-pa

D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-pa

D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-pa

...

D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-pa

D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-pa

D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-pa

D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-pa

D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-pa

In [4]:
offer_sms = infrastructure.get_smartsheet('offers_sms')
offer_sms = offer_sms[offer_sms['Hitpath ID'].isna() == False]
current_date = dt.date.today()  # Get the current date
max_date = current_date - dt.timedelta(days=1)
last_7_days = max_date - dt.timedelta(days = 7)
last_7d_df = all_send_strategy_df[(all_send_strategy_df['Date'].dt.date< max_date ) & (all_send_strategy_df['Date'].dt.date>=  last_7_days) ]
zero_ot_email_body = "* Here are Pipeline Testing Offers with $0 Revenue, please check whether there's tracking issue:\n"
zero_ot_email_body += "\tTableau Link: https://tableau.rxmg.app/#/views/RXMG-SMSReports/0DropsTable?:iid=1"+"\n\n"
last_7d_df['Hitpath Offer ID'] = last_7d_df['Hitpath Offer ID'].astype(str).str.split('.',expand = True)[0]
temp = last_7d_df.loc[last_7d_df['Send Strategy'] == 'PT',].groupby('Hitpath Offer ID')['Revenue'].sum()
zero_drops = temp[temp == 0].reset_index(name = 'Revenue')
if len(zero_drops) != 0 : 
    
    offer_sms['Hitpath ID'] = offer_sms['Hitpath ID'].astype(str).str.split('.',expand = True)[0]
    zero_drops = zero_drops.merge(offer_sms[['Hitpath ID','Scheduling Name']], how = 'left',left_on = 'Hitpath Offer ID',right_on = 'Hitpath ID')
    for i in zero_drops['Hitpath Offer ID'].unique(): 
        scheduling_name = zero_drops[zero_drops['Hitpath Offer ID']==i]['Scheduling Name'].values[0]
        zero_ot_email_body += '\tOffer '+scheduling_name+":\n\tHere are drops with $0 Revenue:\n"
        
        izero_drops = last_7d_df.loc[last_7d_df['Hitpath Offer ID'] == i, ] 
        for j in range(0,len(izero_drops)): 
            temp_df = izero_drops.iloc[j,:]
            date_temp_df =temp_df['Date'].strftime("%Y-%m-%d")
            dp_pub_temp_df = temp_df['DP&Pub']
            zero_ot_email_body += '\t  '+date_temp_df+'\t'+dp_pub_temp_df+"\n"
else: 
    zero_ot_email_body += "\tThere's no $0 Revenue pipeline testing offers in the last 7 days.\n"

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_10856\3978378866.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_7d_df['Hitpath Offer ID'] = last_7d_df['Hitpath Offer ID'].astype(str).str.split('.',expand = True)[0]


In [5]:
# last 10 days 
df_10 = df[df['Date'] > (dt.datetime.today()-timedelta(days=7))]  
# offer eCPM is trend down 50% compared to last drop or 25% compared to last 5 drops 
df_10['Opportunity Cost on offer-level'] =  df_10['Revenue'] - df_10['Offer eCPM recent median'] *  df_10['Delivered'] / 1000
df_10['Offer Alert']=((df_10['Offer eCPM recent slope']<-.5)|(df['Offer eCPM recent median slope']<-.25) | (df['Opportunity Cost eCPM on dataset-level'] < -3))
df_offer_alert = df_10[df_10['Offer Alert']==True]
df_offer_alert['Alert Message'] = ''
df_offer_alert.loc[(df['Opportunity Cost eCPM on dataset-level'] < -3), 'Alert Message'] += "Below Average   "
df_offer_alert.loc[(df_10['Offer eCPM recent slope']<-.5)|(df['Offer eCPM recent median slope']<-.25), 'Alert Message'] += 'Downtrending'
df_offer_alert['Opportunity Cost on dataset-level'] =  df_offer_alert['Opportunity Cost'] 

# find out the percentage of True offer alert by Offers 
summary = df_10.groupby(['Hitpath Offer ID']).agg({'Offer Alert':'sum', 'Datestring':'count','Revenue':'sum','Opportunity Cost on offer-level':'sum','Delivered':'sum','Cost': 'sum'}).reset_index() 
summary['Percentage of Alert'] = summary['Offer Alert'] / summary['Datestring']
summary['eCPM']=summary['Revenue']*1000/summary['Delivered']
# aggregate SC_DP&Pub into string if Offer Alert is True
summary2 = df_offer_alert.groupby(['Hitpath Offer ID']).agg({'SC_DP&Pub':', '.join}).reset_index()
summary3 = summary.merge(summary2, on='Hitpath Offer ID', how='left')
summary3['sorting_score'] = summary3['Percentage of Alert'] * summary3['Offer Alert'] * abs( summary3['Opportunity Cost on offer-level'] / summary3['Revenue'])
summary3 = summary3.sort_values(by='sorting_score', ascending=False)
summary3 = summary3[summary3['Percentage of Alert']>0.3]
summary3.rename(columns={'Offer Alert':'No.Underperforming Drops','Datestring':'Total Drops' }, inplace=True)

# import offers_sms
offers_sms = infrastructure.get_smartsheet('offers_sms')
# merge offers_sms to get scheduling name
summary3['Hitpath Offer ID'] = summary3['Hitpath Offer ID'].astype(str).str.split('.',expand=True)[0]
offers_sms['Hitpath ID'] = offers_sms['Hitpath ID'] .astype(str).str.split('.',expand=True)[0]
summary4 = summary3.merge(offers_sms[['Hitpath ID','Scheduling Name']], left_on = 'Hitpath Offer ID', right_on ='Hitpath ID', how='left', copy = False)
# drop sorting_score
summary4.drop(columns=['sorting_score','Hitpath ID'], inplace=True)
# organize columns 
summary4 = summary4[['Hitpath Offer ID','Scheduling Name','No.Underperforming Drops','Total Drops','Percentage of Alert','Revenue','eCPM','Delivered','Cost','Opportunity Cost on offer-level','SC_DP&Pub']]

# drop clicks and jump page clicks 
df_offer_alert.drop(columns=['Clicks','Jump Page Clicks','Opportunity Cost','Optout','Datestring'], inplace=True)
# order by SC_DP&Pub and Date
df_offer_alert.sort_values(by=['SC_DP&Pub','Date'], inplace=True)
#  organize columns 
df_offer_alert =df_offer_alert[['SC_DP&Pub', 'Hitpath Offer ID', 'Date', 'Send Strategy','Alert Message', 'eCPM','Revenue',\
        'Opportunity Cost on dataset-level', 'Opportunity Cost eCPM on dataset-level',\
        'Offer eCPM recent median','Opportunity Cost on offer-level','Offer eCPM recent slope',\
       'Offer eCPM recent median slope','Delivered','optout rate', 'CTR', 
       'Offer recent cslope', 'Offer CTR recent median slope',\
       'JPCTR', 'Offer recent jpcslope', 'Offer jpctr recent median slope'\
       ]]

for i in df_offer_alert.columns:
        if df_offer_alert[i].dtype == 'float64':
            df_offer_alert[i] = df_offer_alert[i].fillna(0.00)
# change Date to date format
df_offer_alert['Date'] = pd.to_datetime(df_offer_alert['Date']).dt.date


D:\Users\lilig\AppData\Local\Temp\1\ipykernel_10856\3432330665.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['Opportunity Cost on offer-level'] =  df_10['Revenue'] - df_10['Offer eCPM recent median'] *  df_10['Delivered'] / 1000
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_10856\3432330665.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['Offer Alert']=((df_10['Offer eCPM recent slope']<-.5)|(df['Offer eCPM recent median slope']<-.25) | (df['Opportunity Cost eCPM on dataset-level'] < -

In [6]:
filename = filepath.output_folder+'Offer Alert - {}.xlsx'.format(date.today().strftime("%m_%d_%Y"))
writer=pd.ExcelWriter(filename, engine='xlsxwriter')
workbook=writer.book
worksheet = workbook.add_worksheet('Downtrending Offers')
worksheet2 = workbook.add_worksheet('Downtrending Drops') 
# header need wrap_text
header_fmt = workbook.add_format({'bold': 1,'align': 'center','valign': 'vcenter','text_wrap': True})
percent_fmt = workbook.add_format({'num_format': '0.00%'}) 
schedulename_fmt = workbook.add_format({'valign': 'vcenter','text_wrap': True})
dollar_fmt = workbook.add_format({'num_format': '$#,##0.00'})
no_fmt = workbook.add_format({'num_format': '#,##0'})
# write summary4 into worksheet 
worksheet.write(0,0,'Hitpath Offer ID',header_fmt)
worksheet.write(0,1,'Scheduling Name',header_fmt)
worksheet.write(0,2,'No.Underperforming Drops',header_fmt)
worksheet.write(0,3,'Total Drops',header_fmt)
worksheet.write(0,4,'Percentage of Alert',header_fmt)
worksheet.write(0,5,'Revenue',header_fmt)
worksheet.write(0,6,'eCPM',header_fmt)
worksheet.write(0,7,'Delivered',header_fmt)
worksheet.write(0,8,'Cost',header_fmt)
worksheet.write(0,9,'Opportunity Cost on offer-level',header_fmt)
worksheet.write(0,10,'SC_DP&Pub',header_fmt)


summary4.to_excel(writer, sheet_name='Downtrending Offers', startrow=1, startcol=0, index=False, header=False)
worksheet.set_column(0,10,15)
worksheet.set_column(4,4,15,percent_fmt)
worksheet.set_column(6,6,15,dollar_fmt)
worksheet.set_column(8,8,15,dollar_fmt)
worksheet.set_column(9,9,15,dollar_fmt)
worksheet.write_comment('E1','Percentage of Alert = No.Underperforming Drops / Total Drops')
worksheet.write_comment('J1','The opportunity cost we lost/gain compared to the offer median performance')
worksheet.set_column(5,5,15,dollar_fmt)

worksheet.set_column(1,1,40,schedulename_fmt)

# write df_offer_alert into worksheet2
df_offer_alert.to_excel(writer, sheet_name='Downtrending Drops', startrow=1, startcol=0, index=False, header = False)
# set first row into header format
for col_num, value in enumerate(df_offer_alert.columns.values):
    worksheet2.write(0, col_num, value, header_fmt)
# set column width
worksheet2.set_column(0,0,20)
worksheet2.set_column(2,2,15)
worksheet2.set_column(1,20,15)
worksheet2.set_column(3,3,15)
# set revenue, opportunity cost, eCPM, offer eCPM recent median as dollar format 
worksheet2.set_column(5,10,15,dollar_fmt)
# set delivered, CTR, optout rate, JPCTR, slope as percentage format
worksheet2.set_column(11,12,15,percent_fmt)
worksheet2.set_column(14,20,15,percent_fmt)
# set delivered as no format
worksheet2.set_column(13,13,15,no_fmt)
worksheet2.set_column(4,4,20,schedulename_fmt)
# set conditional format for alert message 

bg_yellow =  workbook.add_format({'bg_color':   '#FCF3CF'})
bg_red =  workbook.add_format({'bg_color':   '#F5B7B1'})
bg_orange =  workbook.add_format({'bg_color':   '#F5CBA7'}) 
worksheet2.conditional_format(0,4,df_offer_alert.shape[0],4, {'type':'text','criteria':'containing','value':'Below Average   Downtrending','format':bg_red})
worksheet2.conditional_format(0,4,df_offer_alert.shape[0],4, {'type':'text','criteria':'containing','value':'Downtrending','format':bg_yellow})
worksheet2.conditional_format(0,4,df_offer_alert.shape[0],4, {'type':'text','criteria':'containing','value':'Below Average','format':bg_orange})
worksheet2.write_comment('H1',"The opportunity cost we lost/gain compared to the datasets' average performance in the last 30 days")
worksheet2.write_comment('I1',"The opportunity cost eCPM we lost/gain compared to the datasets' average performance in the last 30 days")
worksheet2.write_comment('J1',"Offer's median eCPM in the last 5 drops in the same account")
worksheet2.write_comment('K1',"The opportunity cost we lost/gain compared to the offer median performance")
worksheet2.write_comment('L1',"The eCPM change rate compared to the eCPM of the last drop")
worksheet2.write_comment('M1',"The eCPM change rate compared to the median eCPM of the last 5 drops")
worksheet2.write_comment('Q1',"The CTR change rate compared to the CTR of the last drop")
worksheet2.write_comment('R1',"The CTR change rate compared to the median CTR of the last 5 drops")
worksheet2.write_comment('S1',"Jump Page CTR")
worksheet2.write_comment('T1',"The jump page CTR change rate compared to the jump page CTR of the last drop")
worksheet2.write_comment('U1',"The jump page CTR change rate compared to the median jump page CTR of the last 5 drops")


workbook.close() 

In [7]:
os.chdir( filepath.output_folder) 
filename = 'Offer Alert - {}.xlsx'.format(date.today().strftime("%m_%d_%Y"))

toaddr = ['sms@rxmg.com','offernotices@rxmg.com','lili@rxmg.com']
#toaddr = ['lili@rxmg.com']
today = date.today().strftime("%m_%d_%Y")
subject_line = f"SMS Team Offer Alert Report - {today}"
email_body = "Hi Team,\n\n"
email_body += "The offer or drop receive the alert once it meet either one of the 3 conditions in the last 7 days:\n\t* The offer decreased by 50% from the last drop in the account;\n\t* The offer decreased by 25% from the median eCPM of the last 5 drops in the account;\n\t* The drop's eCPM is below average dataset eCPM by -$3.\n Please find the attached Offer Alert Report and optimize your scheduling.\n\n" 
email_body += zero_ot_email_body + "\n\n"
email_body += '\n\nHave a nice day!\n\nThanks\nLili Guo\n\n'

for i in toaddr:
    send_email.send_email([filename],subject_line,email_body,i)
print(toaddr)

connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP b14-20020aa7810e000000b006be17e60708sm1723824pfi.204 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP b14-20020aa7810e000000b006be17e60708sm1723824pfi.204 - gsmtp'
connect: b'smtp.gmail.com ESMTP b14-20020aa7810e000000b006be17e60708sm1723824pfi.204 - gsmtp'
send: 'ehlo WSAMZN-G2LI17MA.corp.rxmg.tech\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [52.38.54.186]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo WSAMZN-G2LI17MA.corp.rxmg.tech\r\n'
r

reply: b'250 2.0.0 OK  1699037513 b14-20020aa7810e000000b006be17e60708sm1723824pfi.204 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1699037513 b14-20020aa7810e000000b006be17e60708sm1723824pfi.204 - gsmtp'
data: (250, b'2.0.0 OK  1699037513 b14-20020aa7810e000000b006be17e60708sm1723824pfi.204 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection b14-20020aa7810e000000b006be17e60708sm1723824pfi.204 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection b14-20020aa7810e000000b006be17e60708sm1723824pfi.204 - gsmtp'
connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP f13-20020a170902e98d00b001c9ba6c7287sm1690495plb.143 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP f13-20020a170902e98d00b001c9ba6c7287sm1690495plb.143 - gsmtp'
connect: b'smtp.gmail.com ESMTP f13-20020a170902e98d00b001c9ba6c7287sm1690495plb.143 - gsmtp'
send: 'ehlo WSAMZN-G2LI17MA.corp.rxmg.tech\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186

reply: b'250 2.0.0 OK  1699037514 f13-20020a170902e98d00b001c9ba6c7287sm1690495plb.143 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1699037514 f13-20020a170902e98d00b001c9ba6c7287sm1690495plb.143 - gsmtp'
data: (250, b'2.0.0 OK  1699037514 f13-20020a170902e98d00b001c9ba6c7287sm1690495plb.143 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection f13-20020a170902e98d00b001c9ba6c7287sm1690495plb.143 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection f13-20020a170902e98d00b001c9ba6c7287sm1690495plb.143 - gsmtp'
connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP z16-20020a170903019000b001b9dab0397bsm1694149plg.29 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP z16-20020a170903019000b001b9dab0397bsm1694149plg.29 - gsmtp'
connect: b'smtp.gmail.com ESMTP z16-20020a170903019000b001b9dab0397bsm1694149plg.29 - gsmtp'
send: 'ehlo WSAMZN-G2LI17MA.corp.rxmg.tech\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r

['sms@rxmg.com', 'offernotices@rxmg.com', 'lili@rxmg.com']


reply: b'250 2.0.0 OK  1699037515 z16-20020a170903019000b001b9dab0397bsm1694149plg.29 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1699037515 z16-20020a170903019000b001b9dab0397bsm1694149plg.29 - gsmtp'
data: (250, b'2.0.0 OK  1699037515 z16-20020a170903019000b001b9dab0397bsm1694149plg.29 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection z16-20020a170903019000b001b9dab0397bsm1694149plg.29 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection z16-20020a170903019000b001b9dab0397bsm1694149plg.29 - gsmtp'
